In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
from pennylane import BasisState, FermionicSingleExcitation, FermionicDoubleExcitation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1



In [2]:
def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

     # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("lightning.qubit", wires=qubits)
    else:
        dev = qml.device("lightning.qubit", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        qml.BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                qml.FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                qml.FermionicSingleExcitation(param_array[layer][j], wires=s_w)
        return qml.expval(H)
        #return [qml.expval(op) for op in ops_meas]
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state)

    print(f"UCCSD: {energy:.8f} Ha")
    return params, pool


r = 1.88973
#symbols = ['H', 'H', 'H', 'H']
symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r]
    #[0.0, 1.5*r, 0.0],
    #[0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 2
charge = 0

active_electrons = 2
active_orbitals = 2

#params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)

In [ ]:
def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,p=0, shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
  
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
   
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        for w in wires:
            qml.BitFlip(p=p, wires=w)
        return qml.expval(H)
        

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        for w in wires:
            qml.BitFlip(p=p, wires=w)
        return qml.expval(H)

    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i]= circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
              
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]= Mtmp-M[i,i]/2.0-M[j,j]/2.0
    print("M", M)
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    print('UCCSD_qscEOM_bitflip:', values[0][0])
    return values
#eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.02)

#print('UCCSD_qscEOM_bitflip:', eig[0][0])

In [ ]:
def gs_error(symbols, geometry, electrons, charge, shots=None, max_iter=100, p=0):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", charge=charge, method="pyscf")

     # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
   
    # Device
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)

    
    @qml.qnode(dev, interface="autograd")
    def circuit(params, wires, s_wires, d_wires, hf_state):
        # Initialize HF state
        qml.BasisState(hf_state, wires=wires)
        # Ensure params shape compatible
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)

        # Apply UCCSD excitations
        for layer in range(1):
            # Double excitations
            for i, (w1, w2) in enumerate(d_wires):
                qml.FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
            # Single excitations
            for j, s_w in enumerate(s_wires):
                qml.FermionicSingleExcitation(param_array[layer][j], wires=s_w)
            for w in wires:
                qml.BitFlip(p=p, wires=w)
        return qml.expval(H)
    
    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state)

    print(f"UCCSD_bitflip: {energy} Ha")
    return params, pool

#params, pool = gs_error(symbols, geometry, electrons, charge, max_iter=200, p=0.02)

In [6]:

params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)
params, pool = gs_error(symbols, geometry, electrons, charge, max_iter=200, p=0.9)
eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params, p=0.9)


UCCSD: -0.99814886 Ha
UCCSD_bitflip: -0.8482245841832077 Ha
M [[-8.48224584e-01 -2.77555756e-16 -2.22044605e-16 -4.99600361e-16]
 [-2.77555756e-16 -6.00114445e-01 -1.46903126e-01  1.11022302e-16]
 [-2.22044605e-16 -1.46903126e-01 -6.00114445e-01  2.77555756e-16]
 [-4.99600361e-16  1.11022302e-16  2.77555756e-16 -3.41280634e-01]]
UCCSD_qscEOM_bitflip: -0.8482245841832077


M [[-9.98078263e-01  0.00000000e+00  3.88578059e-16  6.98389859e-03]
 [ 0.00000000e+00 -6.61049049e-01 -2.29536134e-01  3.60822483e-16]
 [ 3.88578059e-16 -2.29536134e-01 -6.61049049e-01  3.05311332e-16]
 [ 6.98389859e-03  3.88578059e-16  3.33066907e-16 -3.07264099e-01]]